In [1]:
import numpy as np
import sys

def generate_gel_slab(beads_per_chain=5, units_x=8, units_y=8, units_z=4, 
                                   solvent_density=0.6,
                                   solvent_padding=5.0, support_thickness=2.0, 
                                   support_spacing=0.4, output_file="gel_slab.data"):
    """
    Generate a tetrahedral (diamond lattice) polymer gel slab in solvent bath.
    
    Parameters:
    - beads_per_chain: number of beads per chain
    - units_x, units_y: number of unit cells in x and y (horizontal)
    - units_z: number of unit cells in z (height, smaller)
    - solvent_density: solvent density throughout entire box (beads/σ³)
    - solvent_padding: thickness of solvent layer around gel (in σ)
    - support_spacing: spacing between support atoms (smaller = denser)
    - output_file: LAMMPS data file name
    """
    
    # Lattice parameters
    bead_spacing = 1.0  # Bond length
    chain_length = bead_spacing * (beads_per_chain - 1)
    z_clearance = 0  # Adjust this value as needed
    
    # Diamond lattice: use chain_length as the FCC lattice constant
    a = chain_length  # FCC lattice constant
    
    # Gel dimensions
    gel_x = units_x * a
    gel_y = units_y * a
    gel_z = units_z * a
    
    # Box dimensions (gel + solvent padding + support)
    box_x = gel_x + 2 * solvent_padding
    box_y = gel_y + 2 * solvent_padding
    box_z = gel_z + solvent_padding + support_thickness + z_clearance
    
    # Offset to center gel in x-y, place above support in z
    offset = np.array([solvent_padding, solvent_padding, 2*support_thickness + z_clearance])
    
    particles = []
    bonds = []
    particle_id = 1
    bond_id = 1
    molecule_id = 1
    
    # Generate diamond lattice crosslinks
    # Diamond = 2 interpenetrating FCC lattices offset by (a/4, a/4, a/4)
    crosslinks = {}
    
    for i in range(units_x + 1):
        for j in range(units_y + 1):
            for k in range(units_z + 1):
                # First FCC sublattice
                fcc1_positions = [
                    np.array([0.0, 0.0, 0.0]),
                    np.array([0.5, 0.5, 0.0]),
                    np.array([0.5, 0.0, 0.5]),
                    np.array([0.0, 0.5, 0.5])
                ]
                
                for sublattice in [0, 1]:  # Two FCC sublattices
                    for fcc_idx, fcc_frac in enumerate(fcc1_positions):
                        if sublattice == 1:
                            # Second sublattice offset by (1/4, 1/4, 1/4)
                            frac_pos = fcc_frac + np.array([0.25, 0.25, 0.25])
                        else:
                            frac_pos = fcc_frac
                        
                        pos = (np.array([i, j, k]) + frac_pos) * a + offset
                        
                        # Only keep if within gel bounds
                        if (0 <= pos[0] - offset[0] <= gel_x and 
                            0 <= pos[1] - offset[1] <= gel_y and 
                            0 <= pos[2] - offset[2] <= gel_z):
                            
                            key = (i, j, k, sublattice, fcc_idx)
                            crosslinks[key] = particle_id
                            particles.append({
                                'id': particle_id,
                                'type': 1,  # Crosslink
                                'pos': pos.copy(),
                                'mol': 0
                            })
                            particle_id += 1
    
    # Generate tetrahedral bonds
    # In diamond lattice, each atom connects to 4 nearest neighbors at distance a*sqrt(3)/4
    bond_distance = a * np.sqrt(3) / 4
    
    created_bonds = set()  # Track to avoid duplicates
    
    for key1, id1 in crosslinks.items():
        pos1 = particles[id1 - 1]['pos']
        
        # Find all neighbors within bond distance
        for key2, id2 in crosslinks.items():
            if id1 >= id2:  # Only create each bond once
                continue
            
            pos2 = particles[id2 - 1]['pos']
            dist = np.linalg.norm(pos2 - pos1)
            
            # Check if this is a nearest neighbor (allow some tolerance)
            if abs(dist - bond_distance) < 0.1 * bond_distance:
                bond_pair = tuple(sorted([id1, id2]))
                
                if bond_pair not in created_bonds:
                    created_bonds.add(bond_pair)
                    
                    # Create chain between crosslinks
                    chain_ids = [id1]
                    
                    for b in range(1, beads_per_chain - 1):
                        frac = b / (beads_per_chain - 1)
                        pos = pos1 * (1 - frac) + pos2 * frac
                        particles.append({
                            'id': particle_id,
                            'type': 2,  # Chain bead
                            'pos': pos.copy(),
                            'mol': molecule_id
                        })
                        chain_ids.append(particle_id)
                        particle_id += 1
                    
                    chain_ids.append(id2)
                    
                    # Assign molecule IDs to crosslinks
                    particles[id1 - 1]['mol'] = molecule_id
                    particles[id2 - 1]['mol'] = molecule_id
                    
                    # Create bonds
                    for b in range(len(chain_ids) - 1):
                        bonds.append({
                            'id': bond_id,
                            'type': 1,
                            'atom1': chain_ids[b],
                            'atom2': chain_ids[b + 1]
                        })
                        bond_id += 1
                    
                    molecule_id += 1
    
    # Create dense support layers with hexagonal packing
    nx = int(box_x / support_spacing) + 1
    ny = int(box_y / (support_spacing * np.sqrt(3))) + 1
    
    # Bottom support layer (type 4 - frozen)
    for i in range(nx):
        for j in range(ny):
            x = i * support_spacing
            y = j * support_spacing * np.sqrt(3) + (support_spacing / 2 if i % 2 else 0)
            z = support_thickness / 2 + z_clearance
            
            if 0 <= x <= box_x and 0 <= y <= box_y:
                particles.append({
                    'id': particle_id,
                    'type': 4,  # Bottom support
                    'pos': np.array([x, y, z]),
                    'mol': molecule_id
                })
                particle_id += 1
                molecule_id += 1
    
    # Top piston layer (type 5 - mobile, for compression)
    for i in range(nx):
        for j in range(ny):
            x = i * support_spacing
            y = j * support_spacing * np.sqrt(3) + (support_spacing / 2 if i % 2 else 0)
            z = box_z - support_thickness / 2
            
            if 0 <= x <= box_x and 0 <= y <= box_y:
                particles.append({
                    'id': particle_id,
                    'type': 5,  # Top piston
                    'pos': np.array([x, y, z]),
                    'mol': molecule_id
                })
                particle_id += 1
                molecule_id += 1
    
    # Add solvent uniformly throughout entire box
    total_volume = box_x * box_y * box_z
    num_solvent_total = int(solvent_density * total_volume)
    
    np.random.seed(42)
    solvent_count = 0
    max_attempts = num_solvent_total * 100
    attempts = 0
    
    while solvent_count < num_solvent_total and attempts < max_attempts:
        attempts += 1
        pos = np.array([
            np.random.rand() * box_x,
            np.random.rand() * box_y,
            np.random.rand() * box_z
        ])
        
        min_dist = 0.8
        too_close = False
        for p in particles[-min(100, len(particles)):]:
            if np.linalg.norm(pos - p['pos']) < min_dist:
                too_close = True
                break
        
        if not too_close:
            particles.append({
                'id': particle_id,
                'type': 3,  # Solvent
                'pos': pos.copy(),
                'mol': molecule_id
            })
            particle_id += 1
            molecule_id += 1
            solvent_count += 1
    
    if solvent_count < num_solvent_total:
        print(f"Warning: Only placed {solvent_count}/{num_solvent_total} solvent particles")
    
    # Write LAMMPS data file
    with open(output_file, 'w') as f:
        f.write("LAMMPS data file for tetrahedral gel slab with piston\n\n")
        f.write(f"{len(particles)} atoms\n")
        f.write(f"{len(bonds)} bonds\n")
        f.write("0 angles\n")
        f.write("0 dihedrals\n")
        f.write("0 impropers\n\n")
        f.write("5 atom types\n")
        f.write("1 bond types\n\n")
        f.write(f"0.0 {box_x} xlo xhi\n")
        f.write(f"0.0 {box_y} ylo yhi\n")
        f.write(f"0.0 {box_z} zlo zhi\n\n")
        f.write("Masses\n\n")
        f.write("1 1.0  # Crosslink\n")
        f.write("2 1.0  # Chain bead\n")
        f.write("3 1.0  # Solvent\n")
        f.write("4 1.0  # Bottom support (frozen)\n")
        f.write("5 1.0  # Top piston (mobile)\n\n")
        f.write("Atoms\n\n")
        
        for p in particles:
            f.write(f"{p['id']} {p['mol']} {p['type']} {p['pos'][0]:.6f} {p['pos'][1]:.6f} {p['pos'][2]:.6f}\n")
        
        f.write("\nBonds\n\n")
        for b in bonds:
            f.write(f"{b['id']} {b['type']} {b['atom1']} {b['atom2']}\n")
    
    num_solvent_final = sum(1 for p in particles if p['type'] == 3)
    num_polymer = sum(1 for p in particles if p['type'] in [1, 2])
    num_bottom_support = sum(1 for p in particles if p['type'] == 4)
    num_top_piston = sum(1 for p in particles if p['type'] == 5)
    num_crosslinks = len(crosslinks)
    num_chains = len(bonds) // (beads_per_chain - 1) if beads_per_chain > 1 else 0
    
    print(f"Generated tetrahedral gel slab with piston:")
    print(f"  Unit cells: {units_x} x {units_y} x {units_z}")
    print(f"  Beads per chain: {beads_per_chain}")
    print(f"  Gel dimensions: {gel_x:.2f} x {gel_y:.2f} x {gel_z:.2f}")
    print(f"  Box dimensions: {box_x:.2f} x {box_y:.2f} x {box_z:.2f}")
    print(f"  Crosslinks: {num_crosslinks}")
    print(f"  Chains: {num_chains}")
    print(f"  Polymer beads: {num_polymer}")
    print(f"  Bottom support: {num_bottom_support} (spacing: {support_spacing:.2f})")
    print(f"  Top piston: {num_top_piston} (spacing: {support_spacing:.2f})")
    print(f"  Solvent beads: {num_solvent_final} (density: {solvent_density:.2f})")
    print(f"  Total atoms: {len(particles)}")
    print(f"  Total bonds: {len(bonds)}")
    print(f"  Output: {output_file}")

In [2]:
# Inputs
numBeads = 5 # beads per chain
rhoSolv = 1 # solvent density
slabWidth = 15
slabHeight = 10
solventPadding = 0.9*slabWidth
solventThickness = 2
supportSpacing = 0.2

outputFile = "slab_support_5beads_10x10x5_rho6_extra_padding43.data"


generate_gel_slab(numBeads, slabWidth, slabWidth, slabHeight, 
                  rhoSolv, solventPadding, solventThickness, supportSpacing, outputFile)


Generated tetrahedral gel slab with piston:
  Unit cells: 15 x 15 x 10
  Beads per chain: 5
  Gel dimensions: 60.00 x 60.00 x 40.00
  Box dimensions: 87.00 x 87.00 x 55.50
  Crosslinks: 19091
  Chains: 36000
  Polymer beads: 127091
  Bottom support: 109654 (spacing: 0.20)
  Top piston: 109654 (spacing: 0.20)
  Solvent beads: 420079 (density: 1.00)
  Total atoms: 766478
  Total bonds: 144000
  Output: slab_support_5beads_10x10x5_rho6_extra_padding43.data
